# Assignment 3
## Group name: [enter your group name with either the prefix "ID2214-" or "FID3214-"]
### Project members: 
Zuzanna Gorczyca, zgo@kth.se

Alga Nour Elimane, nealga@kth.se

Tse An Shih, tashih@kth.se

### Declaration:
By submitting this assignment, it is hereby declared that all group members listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214), no part of the solution has been provided by someone not listed as a project member above, and no part of the solution has been generated by a system.

It is furthermore declared that the submitted assignment will not be shared during the course, with any individual other than the group members listed above and teachers of the course ID2214/FID3214. In particular, the assignment will not be uploaded to any public repository. The submitted assignment can be shared after the course only if written consent has been provided by the course responsible of ID2214/FID3214.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [16]:
import numpy as np
import pandas as pd
import time
import sklearn
from sklearn.tree import DecisionTreeClassifier

In [17]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"sklearn version: {sklearn.__version__}")

Python version: 3.13.5
NumPy version: 2.1.3
Pandas version: 2.2.3
sklearn version: 1.6.1


## Reused functions from Assignment 1

In [18]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# COLUMN FILTER
def create_column_filter(df):
    df_copy = df.copy()
    col_keep = ["CLASS", "ID"]

    for col in df_copy.columns:
        if col not in col_keep:
            if df_copy[col].dropna().nunique() <= 1:
                df_copy = df_copy.drop(columns=[col])

    return df_copy, df_copy.columns.tolist()


def apply_column_filter(df, column_filter):
    df_copy = df.copy()
    df_copy = df_copy[column_filter]
    return df_copy


# ==============================================================================================


# NORMALIZATION MAPING
def create_normalization(df, normalizationtype):
    df_copy = df.copy()
    normalization = {}
    col_keep = ["CLASS", "ID"]

    numeric_cols = df_copy.select_dtypes(include=["int64", "float64"]).columns
    numeric_cols = [c for c in numeric_cols if c not in col_keep]

    if normalizationtype == "minmax":
        min_value = df_copy[numeric_cols].min()
        max_value = df_copy[numeric_cols].max()
        df_copy[numeric_cols] = (df_copy[numeric_cols] - min_value) / (
            max_value - min_value
        )
        for c in numeric_cols:
            normalization[c] = ("minmax", min_value[c], max_value[c])
    elif normalizationtype == "zscore":
        mean_value = df_copy[numeric_cols].mean()
        std_value = df_copy[numeric_cols].std()
        df_copy[numeric_cols] = (df_copy[numeric_cols] - mean_value) / std_value
        for c in numeric_cols:
            normalization[c] = ("zscore", mean_value[c], std_value[c])
    else:
        print("Use minmax or zscore")
        normalization = None

    return df_copy, normalization


def apply_normalization(df, normalization):
    df_copy = df.copy()
    for col in normalization:
        normalization_type, value_1, value_2 = normalization[col]
        if normalization_type == "minmax":
            df_copy[col] = (df_copy[col] - value_1) / (value_2 - value_1)
            df_copy[col] = df_copy[col].clip(0, 1)

        elif normalization_type == "zscore":
            df_copy[col] = (df_copy[col] - value_1) / value_2

    return df_copy


# ==============================================================================================

# INPUTATION

def create_imputation(df):
    df_copy = df.copy()
    col_keep = ["CLASS", "ID"]
    imputation = {}

    cols = df_copy.select_dtypes(
        include=["int64", "float64", "category", "object"]
    ).columns
    cols = [n for n in cols if n not in col_keep]
    for col in cols:
        if df_copy[col].dtype in ["int64", "float64"]:
            if pd.isna(df_copy[col]).all():
                df_copy[col] = 0
                imputation[col] = 0
            else:
                mean_val = df_copy[col].mean()
                df_copy[col] = df_copy[col].fillna(mean_val)
                imputation[col] = mean_val

        elif df_copy[col].dtype == "O":
            if pd.isna(df_copy[col]).all():
                df_copy[col] = ""
                imputation[col] = ""
            else:
                mode_val = df_copy[col].mode().iloc[0]
                df_copy[col] = df_copy[col].fillna(mode_val)
                imputation[col] = mode_val

        elif df_copy[col].dtype == "category":
            if pd.isna(df_copy[col]).all():
                cat_val = df_copy[col].cat.categories[0]
                df_copy[col] = df_copy[col].fillna(cat_val)
                imputation[col] = cat_val
            else:
                mode_val = df_copy[col].mode().iloc[0]
                df_copy[col] = df_copy[col].fillna(mode_val)
                imputation[col] = mode_val

    return df_copy, imputation


def apply_imputation(df, imputation):
    df_copy = df.copy()
    for col in imputation:
        fill_value = imputation[col]  # directly use the value
        df_copy[col] = df_copy[col].fillna(fill_value)
    return df_copy


# ==============================================================================================


# ONE-HOT MAPPING


def create_one_hot(df):
    df_copy = df.copy()
    one_hot = {}
    category_cols = df_copy.select_dtypes(
        include=["object", "category"]
    ).columns.tolist()
    category_cols_no_ids = [c for c in category_cols if c not in ["ID", "CLASS"]]
    for column in category_cols_no_ids:
        all_cat = set(cat for cat in df_copy[column])
        one_hot[column] = {cat: "" for cat in all_cat}
        for cat in all_cat:
            df_copy[f"{column} {cat}"] = df_copy[column].apply(
                lambda x: 1.0 if cat in x else 0.0
            )
            one_hot[column][cat] = f"{column} {cat}"
        df_copy = df_copy.drop(column, axis=1)

    return df_copy, one_hot


def apply_one_hot(df, one_hot):
    df_copy = df.copy()
    category_cols = df_copy.select_dtypes(
        include=["object", "category"]
    ).columns.tolist()
    category_cols_no_ids = [c for c in category_cols if c not in ["ID", "CLASS"]]
    for kay, value in one_hot.items():
        for cat, new_name in value.items():
            df_copy[new_name] = df_copy[kay].apply(lambda x: 1.0 if cat in x else 0.0)
        df_copy = df_copy.drop(kay, axis=1)

    return df_copy


# ==============================================================================================


# DISCRETIZATION MAPING
def create_bins(df, nobins, bintype="equal-width"):
    df_tmp = df.drop(["CLASS", "ID"], axis=1, errors="ignore").select_dtypes(
        include=["int", "float"]
    )

    binning = {}
    df_out = pd.DataFrame()
    for col in df_tmp.columns:
        if bintype == "equal-width":
            res_tmp, bins_tmp = pd.cut(
                df_tmp[col], nobins, retbins=True, labels=False, duplicates="drop"
            )
        elif bintype == "equal-size":
            res_tmp, bins_tmp = pd.qcut(
                df_tmp[col], nobins, retbins=True, labels=False, duplicates="drop"
            )

        bins_tmp[0] = -np.inf  # Set the first element of the binning list to -np.inf
        bins_tmp[-1] = np.inf  # Set the last element of the binning list to np.inf

        binning[col] = tuple(bins_tmp)

    df_out = apply_bins(df, binning)  # Apply binning
    return df_out, binning


def apply_bins(df, binning):
    df_out = df.copy()
    for col in binning:
        df_out[col] = pd.DataFrame(
            pd.cut(df_out[col], binning[col], labels=False, duplicates="drop")
        ).astype("category")
    return df_out


# ==============================================================================================

def accuracy(df, correctlabels):
    df_copy = df.copy()
    predictions = df_copy.idxmax(axis=1).values
    sum = 0
    for i in range(len(predictions)):
        if predictions[i] == correctlabels[i]:
            sum += 1

    return sum / len(correctlabels)

# ==============================================================================================

def brier_score(df, correctlabels):
    scores = []
    for i, label in enumerate(correctlabels):
        correct_vector = np.zeros(len(df.columns))
        index = np.where(df.columns == label)[0][0]
        correct_vector[index] = 1
        row_score = (df.iloc[i].values - correct_vector) ** 2
        scores.append(row_score)
    return np.sum(scores) / len(df)

# ==============================================================================================

def auc(df, correctlabels):
    classes = df.columns
    total_auc = 0
    for c in classes:
        scores = {}
        for index, score in enumerate(df[c].values):
            if score not in scores:
                scores[score] = [0, 0]  

            if correctlabels[index] == c:
                scores[score][0] += 1  
            else:
                scores[score][1] += 1
        sorted_scores = [scores[s] for s in sorted(scores.keys(), reverse=True)]
        positives = [s[0] for s in sorted_scores]
        negatives = [s[1] for s in sorted_scores]
        tot_pos = sum(positives)
        tot_neg = sum(negatives)
        cum_pos = 0
        auc_class = 0
        for i in range(len(positives)):
            if negatives[i] == 0:
                cum_pos += positives[i]  
            elif positives[i] == 0:  
                auc_class += (cum_pos / tot_pos) * (negatives[i] / tot_neg)
            else:
                auc_class += (cum_pos / tot_pos) * (negatives[i] / tot_neg)
                auc_class += (positives[i] / tot_pos) * (negatives[i] / tot_neg) / 2
                cum_pos += positives[i]

        class_weight = sum(1 for label in correctlabels if label == c) / len(df)
        total_auc += class_weight * auc_class

    return total_auc

## 1. Define the class RandomForest

In [19]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# * Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
# *Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
#
# *Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 


class RandomForest():
    def __init__(self):
        self.column_filter = None 
        self.imputation = None
        self.one_hot =None 
        self.labels = None
        self.model = []
        self.base_learner_list = []

    def fit(self, df, no_trees=100):
        bootstrap_samples_X = []
        bootstrap_samples_Y = []
        column_df, self.column_filter = create_column_filter(df)
        inputation_df, self.imputation = create_imputation(column_df)
        df_processed, self.one_hot = create_one_hot(inputation_df.drop('CLASS', axis=1, errors='ignore'))        
        df_processed['CLASS'] = inputation_df['CLASS']
        Y = df_processed.pop("CLASS").values
        self.labels = sorted(np.unique(Y))        
        X = df_processed.drop(['ID'], axis=1, errors='ignore').values
        n_samples, n_features = X.shape
        max_features = int(np.log2(n_features)) if int(np.log2(n_features)) > 0 else 1
        for i in range(no_trees):
            bootstrap = np.random.choice(n_samples, size=n_samples, replace=True)
            bootstrap_X = X[bootstrap]
            bootstrap_Y = Y[bootstrap]
            tree = DecisionTreeClassifier(max_features=max_features)
            tree.fit(bootstrap_X, bootstrap_Y)
            self.model.append(tree)


    def predict(self, df):
        colmun_df = apply_column_filter(df, self.column_filter)
        inputation_df = apply_imputation(colmun_df, self.imputation)
        onehot_df = apply_one_hot(inputation_df, self.one_hot)
        test_df = onehot_df.drop(['CLASS', 'ID'], axis=1, errors='ignore')
        X = test_df.values
        n_test_samples = X.shape[0]
        n_classes = len(self.labels)
        sum_probabilities = np.zeros((n_test_samples, n_classes))       
        for model in self.model:
            model_probe = model.predict_proba(X)
            sum_probabilities += model_probe
        avg_probabilities = sum_probabilities / len(self.model)
        predictions = pd.DataFrame(avg_probabilities, columns=self.labels)
        return predictions


In [20]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.16 s.
Testing time: 0.04 s.
Accuracy: 0.9123
AUC: 0.9888
Brier score: 0.1803


In [21]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

Accuracy on training set: 1.0000
AUC on training set: 1.0000
Brier score on training set: 0.0224


### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [22]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 

#copy from 1
class RandomForest():
    def __init__(self):
        self.column_filter = None 
        self.imputation = None
        self.one_hot =None 
        self.labels = None
        self.model = []
        self.base_learner_list = []

    def fit(self, df, no_trees=100):
        bootstrap_samples_X = []
        bootstrap_samples_Y = []
        column_df, self.column_filter = create_column_filter(df)
        inputation_df, self.imputation = create_imputation(column_df)
        df_processed, self.one_hot = create_one_hot(inputation_df.drop('CLASS', axis=1, errors='ignore'))        
        df_processed['CLASS'] = inputation_df['CLASS']
        Y = df_processed.pop("CLASS").values
        self.labels = sorted(np.unique(Y))        
        X = df_processed.drop(['ID'], axis=1, errors='ignore').values
        n_samples, n_features = X.shape
        max_features = int(np.log2(n_features)) if int(np.log2(n_features)) > 0 else 1
        for i in range(no_trees):
            bootstrap = np.random.choice(n_samples, size=n_samples, replace=True)
            bootstrap_X = X[bootstrap]
            bootstrap_Y = Y[bootstrap]
            tree = DecisionTreeClassifier(max_features=max_features)
            tree.fit(bootstrap_X, bootstrap_Y)
            self.model.append(tree)

# revised predict()
    def predict(self, df):
        colmun_df = apply_column_filter(df, self.column_filter)
        inputation_df = apply_imputation(colmun_df, self.imputation)
        onehot_df = apply_one_hot(inputation_df, self.one_hot)
        test_df = onehot_df.drop(['CLASS', 'ID'], axis=1, errors='ignore')
        X = test_df.values 
        n_test_samples = X.shape[0]

        #changes
        all_labels = list(self.labels)
        n_classes = len(all_labels)
        label_to_index = {label: idx for idx,label in enumerate(all_labels)}
        
        sum_probabilities = np.zeros((n_test_samples, n_classes)) 

        #changes
        for t in self.model:
            t_probabilities = t.predict_proba(X)
            t_classes = t.classes_
            t_matrix = np.zeros((n_test_samples, n_classes))

            for i, class_label in enumerate(t_classes):
                global_index = label_to_index[class_label]
                t_matrix[:, global_index] = t_probabilities[:, i]

            sum_probabilities += t_matrix
        
        avg_probabilities = sum_probabilities / len(self.model)
        
        #Convert to DataFrame with class labels
        predictions = pd.DataFrame(avg_probabilities, columns=self.labels)
        return predictions

In [23]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.17 s.
Testing time: 0.04 s.
Accuracy: 0.9488
AUC: 0.9717
Brier score: 0.0989


## 2b. Estimate predictive performance using out-of-bag predictions

In [ ]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively. By "aggregated out-of-bag predictions" is here meant the sum of all 
#         predicted probabilities (one sum per class and instance). These sums should be divided by the number of predictions
#         (stored in the vector) in order to obtain a single class probability distribution per training instance. 
#         This distribution is considered to be the out-of-bag prediction for each instance, and e.g., the class that 
#         receives the highest probability for each instance can be compared to the correct label of the instance, 
#         when calculating the accuracy using the out-of-bag predictions.
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector
#
#         For example, assuming that we have two class labels, then we may end up with the following matrix:
#
#         2 4
#         4 4
#         5 0
#         ...
#
#         and the vector (no. of predictions) (6, 8, 5, ...)
#
#         The resulting class probability distributions are:
#
#         0.333... 0.666...
#         0.5 0.5
#         1.0 0
class RandomForest():
    def __init__(self):
        #initialisation 
        self.column_filter = None 
        self.imputation = None
        self.one_hot = None 
        self.labels = None
        self.model = []
        self.oob_acc = None  # the oob accuracy 

    def fit(self, df, no_trees=100):
        # our goal is to train our random forest and comput the oob accuracy :
        #preprocessing first 
        column_df, self.column_filter = create_column_filter(df)
        inputation_df, self.imputation = create_imputation(column_df)
        df_processed, self.one_hot = create_one_hot(inputation_df.drop('CLASS', axis=1, errors='ignore'))        
        df_processed['CLASS'] = inputation_df['CLASS']
        Y = df_processed.pop("CLASS").values
        self.labels = sorted(np.unique(Y))        
        X = df_processed.drop(['ID'], axis=1, errors='ignore').values  
        n_samples, n_features = X.shape
        n_classes = len(self.labels)
        max_features = int(np.log2(n_features)) if int(np.log2(n_features)) > 0 else 1
        
        #hint1 : iniatialisation of oob structure 
        oob_predictions = np.zeros((n_samples, n_classes))
        oob_count = np.zeros(n_samples)
        
        for i in range(no_trees):
            #bootstrap with replacement 
            bootstrap_ind = np.random.choice(n_samples, size=n_samples, replace=True)
            bootstrap_X = X[bootstrap_ind]
            bootstrap_Y = Y[bootstrap_ind]
            
            # Training the tree 
            tree = DecisionTreeClassifier(max_features=max_features)
            tree.fit(bootstrap_X, bootstrap_Y)
            self.model.append(tree)

            # we identify the oob             
            in_bag_ind = np.unique(bootstrap_ind)
            
            all_ind = np.arange(n_samples)
            oob_ind = np.setdiff1d(all_ind, in_bag_ind) # to keep only the index not in boostrap 
            
            # for each instance of oob we make a prediction of this tree 
            if len(oob_ind) > 0:  
                X_oob = X[oob_ind]
                
                proba = tree.predict_proba(X_oob)

                proba_aligned = np.zeros((len(oob_ind), n_classes))
                
                # Mapping  each classe of the tree to it's   index 
                for tree_class_idx, tree_class in enumerate(tree.classes_):
                    global_class_idx = self.labels.index(tree_class)
                    proba_aligned[:, global_class_idx] = proba[:, tree_class_idx]
                
                # adding predictions to the mtric
                oob_predictions[oob_ind] += proba_aligned
                oob_count[oob_ind] += 1
        
        #  Finally calculate  l'OOB Accuracy ===
       
        valid_oob_mask = oob_count > 0
        
        if np.sum(valid_oob_mask) > 0:
            oob_predictions_avg = np.zeros_like(oob_predictions)
            oob_predictions_avg[valid_oob_mask] = (
                oob_predictions[valid_oob_mask] / oob_count[valid_oob_mask, None]
            )
            
            # we take the  classe with highest probability
            oob_predicted_labels = np.argmax(oob_predictions_avg[valid_oob_mask], axis=1)            
            # mapping : label -> index
            label_to_idx = {label: idx for idx, label in enumerate(self.labels)}
            Y_ind = np.array([label_to_idx[y] for y in Y])
            
            # Comparing   OOB predictions  with true  labels
            Y_oob_true = Y_ind[valid_oob_mask]
            correct_predictions = np.sum(oob_predicted_labels == Y_oob_true)
            
            # accuracy
            self.oob_acc = correct_predictions / len(Y_oob_true)
        else:
            
            self.oob_acc = 0.0

    def predict(self, df):

        colmun_df = apply_column_filter(df, self.column_filter)
        inputation_df = apply_imputation(colmun_df, self.imputation)
        onehot_df = apply_one_hot(inputation_df, self.one_hot)
        test_df = onehot_df.drop(['CLASS', 'ID'], axis=1, errors='ignore')
        X = test_df.values
        
        n_test_samples = X.shape[0]
        n_classes = len(self.labels)
        sum_probabilities = np.zeros((n_test_samples, n_classes))       
        
        for model in self.model:
            model_probe = model.predict_proba(X) 
            proba_aligned = np.zeros((n_test_samples, n_classes))
            
            for tree_class_idx, tree_class in enumerate(model.classes_):
                global_class_idx = self.labels.index(tree_class)
                proba_aligned[:, global_class_idx] = model_probe[:, tree_class_idx]
            
            sum_probabilities += proba_aligned
            
        avg_probabilities = sum_probabilities / len(self.model)
        predictions = pd.DataFrame(avg_probabilities, columns=self.labels)
        
        return predictions



In [25]:
# Test your code (leave this part unchanged, except for if auc is undefined)
np.random.seed(55)
train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 0.21 s.
OOB accuracy: 0.9555
Testing time: 0.04 s.
Accuracy: 0.9510
AUC: 0.9741
Brier score: 0.0954


In [26]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.